## Cleaned up basic LeNet exploration.


##### *Starting points : Importing, getting data, normalizing*
`Stuff you need, but is hidden here.`

In [5]:
#Starting points : import everything, use cuda if available
import torch
import math
from torch.nn import functional as F
import dlc_practical_prologue as prologue
import matplotlib.pyplot as plt
%matplotlib inline
from torch import optim
from torch import Tensor
from torch import nn

if torch.cuda.is_available():
    device = torch.device('cuda')
    print("Using : {}".format(device))
else:
    device = torch.device('cpu')
    print("Using : {}".format(device))


Using : cuda


In [3]:
def norm_(train_input,test_input):
    """Function to normalize the input --> done IN PLACE!"""
    mu, std = train_input.mean(), train_input.std()
    train_inputOut = train_input.sub_(mu).div_(std)
    test_inputOut = test_input.sub_(mu).div_(std)
    return train_inputOut, test_inputOut

In [ ]:
#Generating pairs of 14x14 and sending to device. Here default
N=1000
(train_input,train_target,train_classes, \
 test_input,test_target,test_classes) = prologue.generate_pair_sets(N)
train_input = train_input.to(device)
test_input = test_input.to(device)
train_target = train_target.to(device)
test_target = test_target.to(device)
train_classes, test_classes = train_classes.to(device), test_classes.to(device)
train_input,test_input = norm_(train_input,test_input);


#### Model : Convnet

`Conv layer 1 : takes 2x14x14 --> 32x12x12 --> Maxpool --> 32x6x6` 

`Conv Layer 2 : Takes 32x6x6 --> 64x4x4 --> Maxpool 64x2x2`

`FC 1 : View(-1,64*2*2) --> 264 (random number but works)`

`FC 2 : 264-->100 --> FC3 --> 2`

`Using dropout, batchnorm on all hidden layers (FC1, FC2)`

`Softmax as last activation, ReLU for all the others`

In [6]:
#defining model
class PlainConvNet(nn.Module):
    def __init__(self):
        super(PlainConvNet, self).__init__()
        #takes 2x14x14
        #gives 32x12x12, 32x6x6 after maxpool
        self.conv1 = nn.Conv2d(2, 32, kernel_size=3) 
        #takes 32x6x6, gives 64x4x4 then 64x2x2 after maxpool
        self.conv2 = nn.Conv2d(32, 64, kernel_size=3)
        #takes 64*2*2 = 256, gives nb_hidden
        self.fc1 = nn.Linear(2*2*64, 250)
        self.bn1 = nn.BatchNorm1d(250)
        #takes nb_hidden, gives 10 classes
        self.fc2 = nn.Linear(250,100)
        self.bn2 = nn.BatchNorm1d(100)
        self.fc3 = nn.Linear(100, 2)
        self.dropout = nn.Dropout(0.25)
        
    def forward(self, x):
        x = F.relu(F.max_pool2d(self.conv1(x), kernel_size=2,stride=2))
        x = F.relu(F.max_pool2d(self.conv2(x), kernel_size=2,stride=2))
        x = self.dropout(self.bn1(F.relu(self.fc1(x.view(-1, 2*2 * 64)))))
        x = self.dropout(self.bn2(F.relu(self.fc2(x))))
        x = F.softmax(self.fc3(x),dim=1)
        return x 

#### Train_model, compute_nb_errors, run_net

All nets run with : 50 epoch, eta = 9e-2, mini_batch_size = 25 by default. 

Maybe use smaller epochs to run faster (35-40)


*run_net(...) does everything, use only this function, it will call the others. (See params)*

*run_net(...) **RETURNS** the test error as a float, useful for average over multiple runs* 

In [8]:
def train_model(model, train_input, train_target, nb_epochs=50, 
                eta=9e-2, mini_batch_size=25,printTrain = False, graphLoss = False):
    """Trains the model, using CrossEntropyLoss and SGD 
    Model : Architecture to be tested, pytorch.nn.Module
    Train_input : Input tensors Nx2x14x14, N = 1000
    Train_target : Target labels, N, classes = 0 or 1
    Nb_epochs : nb of epochs to train over
    eta : Learning rate
    mini_batch_size : Size of minibatch to be processed"""
    model.train(True)
    criterion = nn.CrossEntropyLoss()
    model.to(device)
    criterion.to(device)
    optimizer = optim.SGD(model.parameters(),lr=eta)
    losses = []
    
    for e in range(nb_epochs):
        for b in range(0, train_input.size(0), mini_batch_size):    
            output = model(train_input.narrow(0, b, mini_batch_size))
            loss = criterion(output, train_target.narrow(0, b, mini_batch_size))
            losses.append(loss)
            model.zero_grad()
            loss.backward()
            optimizer.step()
        if printTrain : 
            print("Epoch : {} :: Train error : {}/{}, {:0f}%".format(e,
            compute_nb_errors(model,train_input,train_target,mini_batch_size),train_target.size(0),
            (100*compute_nb_errors(model,train_input,train_target,mini_batch_size)/train_target.size(0))))
    if graphLoss : 
        plt.plot(losses)
        plt.ylabel('loss')
#--------------------------------------------------------------------------------------------------------#
def compute_nb_errors(model,data_input,data_target,mini_batch_size):
    """std from the séries"""
    nb_errors = 0;
    model.to(device)
    data_input, data_target = data_input.to(device),data_target.to(device)
    for b in range(0,data_input.size(0),mini_batch_size):
        output = model(data_input.narrow(0,b,mini_batch_size))
        _, predicted_classes = torch.max(output, 1)
        for k in range(mini_batch_size):
            if data_target[b+k]!=predicted_classes[k]:
                nb_errors += 1
    
    return nb_errors
#--------------------------------------------------------------------------------------------------------#
def run_net(model,train_input,train_target,
            test_input, test_target,
            nb_epochs = 50,eta=9e-2,mini_batch_size=25,
            printTrain = False,graphLoss = False):
    """"""
    model.to(device)
    print("Model tested : {}".format(str(model)[:str(model).find('(')]))
    print("""Using {} epochs, lr = {:.04f},Mini batch size = {}""".format(nb_epochs,
                                                                          eta,mini_batch_size))
    train_model(model, train_input, train_target,
                nb_epochs, eta, mini_batch_size,printTrain,graphLoss)
    model.train(False)
    train_error = compute_nb_errors(model, train_input, train_target,mini_batch_size) / train_input.size(0) * 100
    test_error = compute_nb_errors(model, test_input, test_target,mini_batch_size) / test_input.size(0) * 100
    print('train_error {:.02f}% test_error {:.02f}% \n'.format(
                train_error,
                test_error
            )
        )
    return float(test_error)